# Notebook para desenvolvimento de um agente que escaneia notícias sobre o Bitcoin (news_scanner)

- Agente usa a biblioteca Tavily para escanear notícias sobre o Bitcoin.
- Eu vou ter dois agentes de noticias, um para coletar notícias diretamente relacionadas ao Bitcoin 
- Outro para coletar notícias relacionadas a criptomoedas e noticias sobre a economia mundial que podem impactar o preço do Bitcoin. 

### Tavily news:

In [1]:
from tavily import TavilyClient
from dotenv import load_dotenv
_ = load_dotenv()

tavily_client = TavilyClient()

In [ ]:
search_response_news = tavily_client.search(query="Bitcoin", topic="news", time_range="day", max_results=2)
search_response_finance = tavily_client.search(query="Bitcoin", topic="finance", time_range="day", max_results=2)
print("News Search Results:")
print(search_response_news)
print("\n\n")
print("Finance Search Results:")
print(search_response_finance)

News Search Results:
{'query': 'Bitcoin', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://longportapp.com/en/news/242305766', 'title': 'At the Bitcoin conference, VanS stated: Bitcoin is a part of the mainstream economy in the United States, and stablecoins will not threaten the US dollar - longportapp.com', 'score': 0.43568602, 'published_date': 'Thu, 29 May 2025 02:37:43 GMT', 'content': 'On May 28, local time, at this year\'s Bitcoin 2025 conference, U.S. Vice President JD Vance emphasized that cryptocurrency (especially Bitcoin) has become a part of the mainstream U.S. economy and "will be here to stay": David Sacks, a top cryptocurrency and AI advisor to the Trump administration, revealed to the media that there are over $200 billion in unregulated stablecoins in the market, and a legal framework "could create trillions of dollars in demand for U.S. Treasury bonds almost overnight." Reports indicate that Trump himself holds $2.9 billion in c

In [7]:
search_response_news['results'][0].keys()

dict_keys(['url', 'title', 'score', 'published_date', 'content'])

In [8]:
search_response_finance['results'][0].keys()

dict_keys(['title', 'url', 'content', 'score', 'raw_content'])

In [28]:
querie = "Bitcoin price analysis"
search_response_finance = tavily_client.search(query=querie, topic="news", time_range="day", max_results=2)

print("Finance Search Results:")
print(search_response_finance)

Finance Search Results:
{'query': 'Bitcoin price analysis', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.kitco.com/news/article/2025-05-29/bitcoin-may-29-daily-chart-alert-bulls-remain-firm-control', 'title': 'Bitcoin May 29 daily chart alert - Bulls remain in firm control - KITCO', 'score': 0.42133808, 'published_date': 'Thu, 29 May 2025 12:18:53 GMT', 'content': 'Bitcoin May 29 daily chart alert - Bulls remain in firm control | Kitco News Kitco News Kitco NEWS has a diverse team of journalists reporting on the economy, stock markets, commodities, cryptocurrencies, mining and metals with accuracy and objectivity. (Kitco News) - Thursday, May 29—June bitcoin futures prices are higher in early U.S. trading Thursday. Follow Jim daily on Kitco.com as he provides both AM and PM roundups and a daily Technical Special. ### Whether 145% or 10%, tariff uncertainty is enough to stop U.S. gold and silver imports, distort the metals market at all lev

In [42]:
extract = tavily_client.extract(urls=["https://www.binance.com/en/square/news/bitcoin-news"])
extract

{'results': [{'url': 'https://www.binance.com/en/square/news/bitcoin-news',
   'raw_content': "Top Bitcoin News News Today | Binance Square\n===============\n\nLogin\nSign Up\nHomeNotificationProfileTrending ArticlesNewsBookmarked and LikedHistoryCreator CenterSettings\nNews\nMay 28 Wed\nOnly Display Breaking News\nAllBinance NewsMarket NewsBitcoin NewsBNB NewsEthereum NewsAI NewsMeme NewsWhale AlertDeFi NewsGameFi NewsWeb3 NewsSolana NewsXRP NewsDOGE NewsSHIB NewsPEPE NewsRegulation NewsNFT NewsEditor's Pick\nLatest Bitcoin news, price updates, and market trends\n5m\n### Meta Urged to Consider Bitcoin for Corporate Strategy According to Odaily, Strive CEO Matt Cole has urged Mark Zuckerberg's Meta to purchase Bitcoin and incorporate it into the company's balance sheet. Cole emphasized that Bitcoin could be beneficial for businesses.\nBTC-2.60%\n0\n0\n2h\n### U.S. Vice President Vance Affirms Continued Bitcoin Holdings According to BlockBeats, U.S. Vice President Vance revealed during 

In [43]:
print(extract['results'][0]['raw_content'])

Top Bitcoin News News Today | Binance Square

Login
Sign Up
HomeNotificationProfileTrending ArticlesNewsBookmarked and LikedHistoryCreator CenterSettings
News
May 28 Wed
Only Display Breaking News
AllBinance NewsMarket NewsBitcoin NewsBNB NewsEthereum NewsAI NewsMeme NewsWhale AlertDeFi NewsGameFi NewsWeb3 NewsSolana NewsXRP NewsDOGE NewsSHIB NewsPEPE NewsRegulation NewsNFT NewsEditor's Pick
Latest Bitcoin news, price updates, and market trends
5m
### Meta Urged to Consider Bitcoin for Corporate Strategy According to Odaily, Strive CEO Matt Cole has urged Mark Zuckerberg's Meta to purchase Bitcoin and incorporate it into the company's balance sheet. Cole emphasized that Bitcoin could be beneficial for businesses.
BTC-2.60%
0
0
2h
### U.S. Vice President Vance Affirms Continued Bitcoin Holdings According to BlockBeats, U.S. Vice President Vance revealed during his speech at the Bitcoin 2025 Conference that he still holds a significant amount of Bitcoin. His statement underscores the ong

In [ ]:
extract2 = tavily_client.extract(urls=["https://www.coindesk.com/search"], extract_depth='advanced')
extract2

### Função que coleta as notícias (modo news):

In [ ]:
import asyncio
from tavily import AsyncTavilyClient

tavily_client = AsyncTavilyClient()

async def fetch_and_extract():
    # Define the queries with search_depth and max_results inside the query dictionary
    queries = [
        {"query": "Bitcoin", "topic": "news", "search_depth": "advanced", "time_range": "day","max_results": 2},
        {"query": "Bitcoin price analysis", "topic": "news", "search_depth": "advanced", "time_range": "day", "max_results": 2},
        {"query": "Bitcoin Fear & Greed Index", "topic": "news", "search_depth": "advanced", "time_range": "week", "max_results": 2},
    ]

    # Perform the search queries concurrently, passing the entire query dictionary
    responses = await asyncio.gather(*[tavily_client.search(**q) for q in queries])

    # Retorna o resultado:
    return responses

# Função que formata as respostas em um novo dicionario estruturado:
def format_search_news(responses):

    search_response = responses
    list_dict_news = []

    for response in search_response:
        for item in response.get('results', []):
            list_dict_news.append({
                'titulo': item.get('title'),
                'url': item.get('url'),
                'data': item.get('published_date'),
                'conteudo': item.get('content'),
                'score': item.get('score'),
            })
    return list_dict_news

In [ ]:
resultado = await fetch_and_extract()
print(resultado)

In [ ]:
result_formatado = format_search_news(resultado)
result_formatado

### Função que coleta e extrai as notícias (modo finance e extract):

In [ ]:
import asyncio
from tavily import AsyncTavilyClient

tavily_client = AsyncTavilyClient()

async def fetch_and_extract():
    # Define the queries with search_depth and max_results inside the query dictionary
    queries = [
        {"query": "Bitcoin", "topic": "finance", "search_depth": "advanced", "time_range": "week", "max_results": 3},
    ]

    # Perform the search queries concurrently, passing the entire query dictionary
    responses = await asyncio.gather(*[tavily_client.search(**q) for q in queries])

    return responses

# Função que recebe a lista de dicionários formatado (com todas as repostas) filtra os scores maiores que 0.5, extrai o conteudo(com tavily_client.extract) e formata o resultado em uma nova lista de dicionarios:
async def filter_and_extract(responses):

    noticias_filtradas = []
    for response in responses:
        for result in response.get('results', []):
            if result.get('score', 0) > 0.5:
                noticias_filtradas.append(result)

    # Para cada uma das notícias filtradas, pegar a url e extrair o conteúdo usando tavily_client.extract, em seguida adicionar a chave 'raw_content' ao dicionário de notícias filtradas... 




# Função que formata as respostas em um novo dicionario estruturado:
def format_search_finance(responses):

    search_response = responses
    list_dict_news = []

    for response in search_response:
        for item in response.get('results', []):
            list_dict_news.append({
                'titulo': item.get('title'),
                'url': item.get('url'),
                'conteudo': item.get('raw_content'),
                'score': item.get('score'),
            })
    return list_dict_news

In [ ]:
# Função de formatação
def formatar_resposta(resposta):
    linhas = []
    for idx, item in enumerate(resposta, start=1):
        title = item.get('title', '').strip()
        url = item.get('url', '').strip()
        date = item.get('published_date', '').strip()
        content = item.get('content', '').strip()

        linhas.append(f"Notícia {idx}:")
        linhas.append(f"Título: {title}")
        linhas.append(f"URL: {url}")
        linhas.append(f"Data de publicação: {date}")
        linhas.append(f"Conteúdo: {content}")
        linhas.append("")  # linha em branco entre notícias

    return "\n".join(linhas)

------------------------------------------------

### Função de coleta de noticias da Openai

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Obter a data atual
today = datetime.now().strftime("%m/%d/%Y")


instruction = """
You are a professional news assistant about Bitcoin.
You are expert in analyzing news and information from the internet to extract the most relevant and up-to-date information.
"""

input_messages = [
    {
        "role": "user", 
        "content": f"""Give me recent Bitcoin news (05/16/2025) based on the following aspects:
        1. Bitcoin price {today}
        2. Bitcoin price action “support” and “resistance” daily analysis {today}.
        3. Bitcoin market sentiment “Crypto Fear and Greed Index” {today} .
        4. Impact of macroeconomic events on Bitcoin price {today}.
        5. Bitcoin buy and sell signals moving average crossover RSI MACD forecast {today}.
        """
    },

]

In [ ]:
from openai import OpenAI
client = OpenAI()
from datetime import datetime
from dateutil.relativedelta import relativedelta


def news_openai():

    today = datetime.now().strftime("%m/%d/%Y")
    this_week = (datetime.now() - relativedelta(weeks=1)).strftime("%m/%d/%Y")
    instruction = """
    You are a professional news assistant about Bitcoin.
    You are expert in analyzing news and information from the internet to extract the most relevant and up-to-date information.
    """

    input_messages = [
        {
            "role": "user", 
            "content": f"""Give me recent Bitcoin news (05/16/2025) based on the following aspects:
            1. Bitcoin price analysis {today}
            2. Bitcoin market sentiment “Crypto Fear and Greed Index” {this_week} - {today}.
            3. Important events that impact Bitcoin price this week {this_week} - {today}.
            4. Macroeconomic outlook May 2025.
            5. Macroeconomic events impact on Bitcoin price {today}.
            6. Bitcoin buy and sell signals {today}.
            """
        },
    ]

    # Create a response using the OpenAI client
    response = client.responses.create(
        model="gpt-4.1",
        instructions=instruction,
        tools=[{
            "type": "web_search_preview",
            "search_context_size": "medium",
        }],
        input=input_messages,
        tool_choice={"type": "web_search_preview"},
    )

    return response

In [ ]:
print(response.model_dump_json(indent=2))

In [ ]:
print(response.output_text)

In [ ]:
def mostra_urls_usadas(response):
    print("URLs citadas:")
    for block in response.output:
        if not hasattr(block, 'content'):
            continue
        for content_item in block.content:
            if not hasattr(content_item, 'annotations'):
                continue
            for annotation in content_item.annotations:
                if annotation.type == "url_citation":
                    print(f"- {annotation.title}: {annotation.url}")

### Teste taapio:

In [ ]:
# Import the requests library 
import requests 
import os
import json

# Define indicator
indicator = "sma"
# Define endpoint 
endpoint = f"https://api.taapi.io/{indicator}"
# Define a parameters dict for the parameters to be sent to the API 
parameters = {
    'secret': os.getenv('TAAPI_IO_API_KEY'),
    'exchange': 'binance',
    'symbol': 'BTC/USDT',
    'interval': '1d',
    'period': '20',
    }  
# Send get request and save the response as response object 
response = requests.get(url = endpoint, params = parameters)
# Extract data in json format 
result = response.json() 
# Print result
print(result)

In [ ]:
# Define indicator
indicator = "sma"
# Define endpoint 
endpoint = f"https://api.taapi.io/{indicator}"
# Define a parameters dict for the parameters to be sent to the API 
parameters = {
    'secret': os.getenv('TAAPI_IO_API_KEY'),
    'exchange': 'binance',
    'symbol': 'BTC/USDT',
    'interval': '1h',
    'period': '20',
    }  
# Send get request and save the response as response object 
response = requests.get(url = endpoint, params = parameters)
# Extract data in json format 
result = response.json() 
# Print result
print(result)

In [ ]:


# Import the requests library 
import requests 
import os
import json

url = "https://api.taapi.io/bulk"
  
# Define a parameters dict for the parameters to be sent to the API 
payload = {
	"secret": os.getenv('TAAPI_IO_API_KEY'),
	"construct": [
		{
			"exchange": "binance",
			"symbol": "BTC/USDT",
			"interval": "1h",
			"indicators": [
				{
					"indicator": "sma",
					"period": 50
				}
			]
		},
		{
			"exchange": "binance",
			"symbol": "BTC/USDT",
			"interval": "4h",
			"indicators": [
				{
					"indicator": "sma",
					"period": 50
				}
			]
		},
		{
			"exchange": "binance",
			"symbol": "BTC/USDT",
			"interval": "1d",
			"indicators": [
				{
					"indicator": "sma",
					"period": 50
				}
			]
		},
        {
			"exchange": "binance",
			"symbol": "BTC/USDT",
			"interval": "1d",
			"indicators": [
				{
					"indicator": "rsi",
					"period": 14
				}
			]
		},
        {
			"exchange": "binance",
			"symbol": "BTC/USDT",
			"interval": "1d",
			"indicators": [
				{
					"indicator": "bbands",
					"period": 20
				}
			]
		}
	]
}
  
headers = {"Content-Type": "application/json"}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

In [ ]:
def indicadores_tecnicos_taapi_io(indicador: str, intervalo: str, periodo: int):
    # Define endpoint 
    endpoint = f"https://api.taapi.io/{indicador}"
    # Define a parameters dict for the parameters to be sent to the API 
    parameters = {
        'secret': os.getenv('TAAPI_IO_API_KEY'),
        'exchange': 'binance',
        'symbol': 'BTC/USDT',
        'interval': intervalo,
        'period': periodo,
        }  
    # Send get request and save the response as response object 
    response = requests.get(url = endpoint, params = parameters)
    # Extract data in json format 
    result = response.json() 
    # Print result
    print(result)

In [ ]:
import time
sma_periods = [20, 50, 100, 200]
for period in sma_periods:
    sma = indicadores_tecnicos_taapi_io("sma", "1h", period)
    time.sleep(15)

### Coindesk api news: ✅

In [34]:
import requests 

response = requests.get('https://data-api.coindesk.com/news/v1/article/list',
    params={"lang":"EN", "limit":10, "categories":"BTC,MARKET", "api_key":"fd8a81d53ce5de8a8cb711d0c99896fe94879d0f4870da333592d712241fb30d"},
    headers={"Content-type":"application/json; charset=UTF-8"}
)

json_response = response.json()

In [35]:
json_response

{'Data': [{'TYPE': '121',
   'ID': 45263761,
   'GUID': 'https://ambcrypto.com/?p=496298',
   'PUBLISHED_ON': 1748559614,
   'IMAGE_URL': 'https://images.cryptocompare.com/news/default/ambcrypto.png',
   'TITLE': 'HYPE’s fate tied to Bitcoin? What whale positions on Hyperliquid say',
   'SUBTITLE': None,
   'AUTHORS': 'Olayiwola Dolapo',
   'URL': 'https://ambcrypto.com/hypes-fate-tied-to-bitcoin-what-whale-positions-on-hyperliquid-say/',
   'SOURCE_ID': 32,
   'BODY': 'Could this liquidity flood pull HYPE back into rally mode?',
   'KEYWORDS': 'Altcoin|News|News 1|Social|Trading View',
   'LANG': 'EN',
   'UPVOTES': 0,
   'DOWNVOTES': 0,
   'SCORE': 0,
   'SENTIMENT': 'NEUTRAL',
   'STATUS': 'ACTIVE',
   'CREATED_ON': 1748559623,
   'UPDATED_ON': None,
   'SOURCE_DATA': {'TYPE': '120',
    'ID': 32,
    'SOURCE_KEY': 'ambcrypto',
    'NAME': 'AMB Crypto',
    'IMAGE_URL': 'https://images.cryptocompare.com/news/default/ambcrypto.png',
    'URL': 'https://ambcrypto.com/feed/',
    'LANG

In [38]:
json_response.keys()

dict_keys(['Data', 'Err'])

In [36]:
noticias_coindesk = []
for item in json_response['Data']:
    noticias_coindesk.append({
        'titulo': item.get('TITLE'),
        'url': item.get('URL'),
        'palavras_chave': item.get('KEYWORDS'),
        'sentimento': item.get('SENTIMENT'),
        'data': item.get('PUBLISHED_ON'),
        'conteudo': item.get('BODY'),
    })

In [39]:
from datetime import datetime
import pytz

def parse_iso8601(date_str):
    """
    Converte string ISO 8601 (UTC) para datetime c/ tzinfo UTC.
    """
    dt = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")
    return dt.replace(tzinfo=pytz.UTC)


def format_datetime(dt, fmt="%d/%m/%Y %H:%M:%S %Z"):  
    """
    Formata datetime em string legível.
    """
    return dt.strftime(fmt)

def extract_unix_api_items(api_response, local_tz=None, output_fmt="%d/%m/%Y %H:%M:%S %Z"):
    """
    Extrai campos (TITLE, URL, KEYWORDS, SENTIMENT, PUBLISHED_ON, BODY) da segunda API.
    Converte PUBLISHED_ON (timestamp UTC em segundos) para formato legível.
    """
    extracted = []
    for entry in api_response['Data']:
        raw_ts = entry.get('PUBLISHED_ON')
        # Converte timestamp UTC em seconds para datetime
        dt = datetime.fromtimestamp(raw_ts, tz=pytz.UTC) if raw_ts else None
        if dt and local_tz:
            dt = dt.astimezone(local_tz)
        formatted = format_datetime(dt, output_fmt) if dt else None

        extracted.append({
            'TITLE': entry.get('TITLE'),
            'URL': entry.get('URL'),
            'KEYWORDS': entry.get('KEYWORDS'),
            'SENTIMENT': entry.get('SENTIMENT'),
            'PUBLISHED_ON': formatted,
            'BODY': entry.get('BODY')
        })
    return extracted

In [40]:
sp_tz = pytz.timezone('America/Sao_Paulo')
result = extract_unix_api_items(json_response, local_tz=sp_tz)

In [41]:
result

[{'TITLE': 'HYPE’s fate tied to Bitcoin? What whale positions on Hyperliquid say',
  'URL': 'https://ambcrypto.com/hypes-fate-tied-to-bitcoin-what-whale-positions-on-hyperliquid-say/',
  'KEYWORDS': 'Altcoin|News|News 1|Social|Trading View',
  'SENTIMENT': 'NEUTRAL',
  'PUBLISHED_ON': '29/05/2025 20:00:14 -03',
  'BODY': 'Could this liquidity flood pull HYPE back into rally mode?'},
 {'TITLE': 'AVAX May Hit $35 Soon, But Ruvi AI (RUVI) Could Deliver 13,233% Returns for Early Investors',
  'URL': 'https://www.cryptopolitan.com/avax-may-hit-35-soon-but-ruvi-ai-ruvi-could-deliver-13233-returns-for-early-investors/',
  'KEYWORDS': 'Press Release',
  'SENTIMENT': 'POSITIVE',
  'PUBLISHED_ON': '29/05/2025 20:00:00 -03',
  'BODY': 'While Avalanche (AVAX) is going to $35, it’s Ruvi AI (RUVI) that’s got the crypto world buzzing with 13,233% returns. With investors looking for exponential growth, Ruvi AI is the game-changer in the blockchain and AI space for those ready to grab the next big oppo

### Coindesk OHLCV:

In [5]:
import requests
import pandas as pd
#import pandas_ta as ta
import os
from dotenv import load_dotenv
import json

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()


def fetch_coindesk_ohlcv(api_key: str, instrument: str = "BTC-USD", limit: int = 250) -> dict | None:
    """
    Busca dados históricos OHLCV da API CoinDesk.

    Args:
        api_key (str): Sua chave de API da CoinDesk.
        instrument (str): O instrumento a ser buscado (ex: "BTC-USD").
        limit (int): O número de dias de dados a serem retornados.
                     Para indicadores como SMA de 200 dias, um limite de pelo menos 200-250 é recomendado.

    Returns:
        dict | None: A resposta JSON da API ou None em caso de erro.
    """
    url = 'https://data-api.coindesk.com/index/cc/v1/historical/days'
    params = {
        "market": "cadli",  # Este mercado parece específico, verifique se é o desejado para BTC-USD geral
        "instrument": instrument,
        "aggregate": 1,
        "fill": "true",
        "apply_mapping": "true",
        "response_format": "JSON",
        "limit": limit,
        "groups": "OHLC,VOLUME", # Garante que estamos pegando OHLC e o campo VOLUME principal
        "api_key": api_key
    }
    headers = {
        "Content-type": "application/json; charset=UTF-8"
    }

    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()  # Levanta um erro HTTP para respostas ruins (4xx ou 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar dados da API CoinDesk: {e}")
        return None

In [6]:
teste = fetch_coindesk_ohlcv(os.getenv('COINDESK_API_KEY'), instrument="BTC-USD", limit=250)

In [7]:
teste

{'Data': [{'UNIT': 'DAY',
   'TIMESTAMP': 1727568000,
   'OPEN': 65887.2135773289,
   'HIGH': 66083.01348839,
   'LOW': 65465.0294142081,
   'CLOSE': 65626.5046907249,
   'VOLUME': 118592.273372256,
   'QUOTE_VOLUME': 7797769947.47553,
   'VOLUME_TOP_TIER': 62938.5640529879,
   'QUOTE_VOLUME_TOP_TIER': 4138370535.90633,
   'VOLUME_DIRECT': 15161.901458264,
   'QUOTE_VOLUME_DIRECT': 997024754.124539,
   'VOLUME_TOP_TIER_DIRECT': 11633.57350217,
   'QUOTE_VOLUME_TOP_TIER_DIRECT': 764945057.404365},
  {'UNIT': 'DAY',
   'TIMESTAMP': 1727654400,
   'OPEN': 65626.5046907249,
   'HIGH': 65634.6632562962,
   'LOW': 62894.9934452219,
   'CLOSE': 63332.7776140374,
   'VOLUME': 383332.54931369,
   'QUOTE_VOLUME': 24530931549.7999,
   'VOLUME_TOP_TIER': 214559.188914789,
   'QUOTE_VOLUME_TOP_TIER': 13726628873.9195,
   'VOLUME_DIRECT': 55085.865351886,
   'QUOTE_VOLUME_DIRECT': 3524434485.35093,
   'VOLUME_TOP_TIER_DIRECT': 43065.34437998,
   'QUOTE_VOLUME_TOP_TIER_DIRECT': 2754835142.43758},
  {

In [ ]:
def parse_coindesk_response_to_dataframe(json_response: dict) -> pd.DataFrame | None:
    """
    Converte a resposta JSON da CoinDesk em um DataFrame do Pandas.

    Args:
        json_response (dict): A resposta JSON da função fetch_coindesk_ohlcv.

    Returns:
        pd.DataFrame | None: Um DataFrame com colunas 'timestamp', 'open', 'high', 'low', 'close', 'volume'
                              e 'timestamp' como índice, ou None se a entrada for inválida.
    """
    if not json_response or 'Data' not in json_response or not json_response['Data']:
        print("Resposta JSON inválida ou sem dados.")
        return None

    data = json_response['Data']
    df = pd.DataFrame(data)

    # Renomear colunas para o padrão (minúsculas) e selecionar as principais
    # A API parece retornar em maiúsculas: OPEN, HIGH, LOW, CLOSE, VOLUME
    column_mapping = {
        'TIMESTAMP': 'timestamp',
        'OPEN': 'open',
        'HIGH': 'high',
        'LOW': 'low',
        'CLOSE': 'close',
        'VOLUME': 'volume' # Usando o campo 'VOLUME' principal conforme discutido
    }
    
    # Filtrar para manter apenas as colunas que existem e que queremos mapear
    existing_columns_to_rename = {k: v for k, v in column_mapping.items() if k in df.columns}
    df = df.rename(columns=existing_columns_to_rename)
    
    # Manter apenas as colunas renomeadas que são relevantes
    final_columns = [col for col in column_mapping.values() if col in df.columns]
    df = df[final_columns]

    if 'timestamp' not in df.columns:
        print("Coluna 'TIMESTAMP' não encontrada nos dados.")
        return None

    # Converter timestamp Unix (em segundos) para datetime e definir como índice
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df = df.set_index('timestamp')
    
    # Converter colunas OHLCV para numérico, caso não estejam
    for col in ['open', 'high', 'low', 'close', 'volume']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Ordenar pelo índice de tempo, do mais antigo para o mais novo, importante para TA
    df = df.sort_index(ascending=True)
            
    return df

In [ ]:
dataframe = parse_coindesk_response_to_dataframe(teste)
dataframe

In [ ]:
def calculate_technical_indicators_v2(df: pd.DataFrame, 
                                     sma_periods: list = [20, 30, 50, 200], 
                                     ema_periods: list = [20], 
                                     rsi_period: int = 14, 
                                     macd_fast: int = 12, 
                                     macd_slow: int = 26, 
                                     macd_signal: int = 9) -> pd.DataFrame | None:
    """
    Calcula indicadores técnicos (SMAs, EMAs, RSI, MACD) usando pandas_ta
    e retorna um NOVO DataFrame contendo apenas esses indicadores.

    Args:
        df (pd.DataFrame): DataFrame com colunas 'open', 'high', 'low', 'close', 'volume'.
                           Deve ter um DatetimeIndex.
        sma_periods (list): Lista de períodos para SMAs.
        ema_periods (list): Lista de períodos para EMAs.
        rsi_period (int): Período para o RSI.
        macd_fast (int): Período rápido para o MACD.
        macd_slow (int): Período lento para o MACD.
        macd_signal (int): Período do sinal para o MACD.

    Returns:
        pd.DataFrame | None: Um novo DataFrame contendo apenas as colunas dos indicadores calculados,
                              compartilhando o mesmo índice do DataFrame de entrada. Retorna None se a entrada for inválida.
    """
    if df is None or df.empty:
        print("DataFrame de entrada (OHLCV) está vazio ou é None.")
        return None
    
    if not isinstance(df.index, pd.DatetimeIndex):
        print("O DataFrame de entrada (OHLCV) deve ter um DatetimeIndex.")
        return None

    # Lista para armazenar todas as séries/dataframes de indicadores
    all_indicators = []

    # Calcular SMAs
    for period in sma_periods:
        sma = df.ta.sma(length=period) # Retorna uma Série, nomeada automaticamente (ex: SMA_50)
        all_indicators.append(sma)

    # Calcular EMAs
    for period in ema_periods:
        ema = df.ta.ema(length=period) # Retorna uma Série, nomeada automaticamente (ex: EMA_20)
        all_indicators.append(ema)

    # Calcular RSI
    rsi = df.ta.rsi(length=rsi_period) # Retorna uma Série, nomeada automaticamente (ex: RSI_14)
    all_indicators.append(rsi)

    # Calcular MACD
    # df.ta.macd() retorna um DataFrame com 3 colunas: MACD_fast_slow_signal, MACDh_..., MACDs_...
    macd_df = df.ta.macd(fast=macd_fast, slow=macd_slow, signal=macd_signal, append=False)
    if macd_df is not None and not macd_df.empty:
        all_indicators.append(macd_df)
    
    # Concatenar todas as séries/dataframes de indicadores em um único DataFrame
    if not all_indicators:
        print("Nenhum indicador foi calculado.")
        return pd.DataFrame(index=df.index) # Retorna DF vazio com o mesmo índice

    indicators_df = pd.concat(all_indicators, axis=1)

    # Remover as linhas com NaN
    indicators_df = indicators_df.dropna()
    
    return indicators_df

In [ ]:
calculos = calculate_technical_indicators_v2(dataframe,
                                          sma_periods=[20, 30, 50, 200], 
                                          ema_periods=[20], 
                                          rsi_period=14, 
                                          macd_fast=12, 
                                          macd_slow=26, 
                                          macd_signal=9)

In [ ]:
calculos

In [ ]:
dataframe

In [ ]:
import pandas_ta as ta

help(ta.bbands)


In [ ]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta


ohlc_df = yf.Ticker('BTC-USD').history(period="3mo", interval="1h")
ohlc_df

In [ ]:
ohlc_df.columns

In [ ]:
ohlc_df_copy = ohlc_df.copy()

In [ ]:
rsi14_4h = ta.rsi(ohlc_df_copy['Close'], length=4)
rsi14_4h

In [ ]:
MyStrategy = ta.Strategy(
    name="MyStrategy",
    ta=[
        {"kind": "sma", "length": 20},
        {"kind": "sma", "length": 30},
        {"kind": "sma", "length": 50},
        {"kind": "sma", "length": 200},
        {"kind": "ema", "length": 20},
        {"kind": "ema", "length": 30},
        {"kind": "ema", "length": 50},
        {"kind": "ema", "length": 200},
        {"kind": "rsi", "length": 14},
        {"kind": "macd", "fast": 12, "slow": 26, "signal": 9},
        {"kind": "bbands", "length": 20, "std": 2},
        {"kind": "adx", "length": 14},
        {"kind": "atr", "length": 14},
        {"kind": "obv"},
    ]
)

In [ ]:
ohlc_df.ta.strategy(MyStrategy)

In [ ]:
ohlc_df.columns

In [ ]:
teste_df = yf.Ticker('BTC-USD').history(period="1mo", interval="4h")

In [ ]:
rsi14_4h = ta.rsi(teste_df['Close'], length=14)
rsi14_4h

### Coleta noticias yfinance: ✅

In [1]:
import yfinance as yf

In [4]:
symbol = 'BTC-USD'
ticker = yf.Ticker(symbol)
# Obtendo noticias recentes
news = ticker.get_news()

In [17]:
news[0].keys()

dict_keys(['id', 'content'])

In [19]:
news[0]['content'].keys()

dict_keys(['id', 'contentType', 'title', 'description', 'summary', 'pubDate', 'displayTime', 'isHosted', 'bypassModal', 'previewUrl', 'thumbnail', 'provider', 'canonicalUrl', 'clickThroughUrl', 'metadata', 'finance', 'storyline'])

In [22]:
from datetime import datetime
import pytz

def parse_iso8601(date_str):
    # Formato ISO 8601 com 'Z' indicando UTC
    dt = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")
    # Atribui timezone UTC
    return dt.replace(tzinfo=pytz.UTC)

def format_datetime(dt, fmt="%d/%m/%Y %H:%M:%S %Z"):  # exemplo: '28/05/2025 13:39:31 UTC'
    return dt.strftime(fmt)

In [23]:
def extract_api_items(api_response, local_tz=None, output_fmt="%d/%m/%Y %H:%M:%S %Z"):
    """
    Extrai de uma lista de respostas de API os campos: title, summary, pubDate e canonicalUrl.

    Args:
        api_response (list): Lista de dicionários retornada pela API.

    Returns:
        list: Lista de dicionários contendo apenas os campos desejados.
    """
    extracted = []
    for entry in api_response:
        content = entry.get('content', {})
        raw_date = content.get('pubDate')
        dt = parse_iso8601(raw_date) if raw_date else None
        # Converte para timezone local se fornecido
        if dt and local_tz is not None:
            dt = dt.astimezone(local_tz)
        formatted = format_datetime(dt, output_fmt) if dt else None

        item = {
            'title': content.get('title'),
            'summary': content.get('summary'),
            'pubDate': formatted,
            'canonicalUrl': content.get('canonicalUrl', {}).get('url')
        }
        extracted.append(item)
    return extracted

In [24]:
extract_api_items(news)

[{'title': 'How bitcoin miners are powering the AI boom',
  'summary': "Some bitcoin (BTC-USD) miners are pivoting to artificial intelligence (AI) as the rapidly evolving tech's data center and power demands expand. BitFarms (BITF) CEO Ben Gagnon joins Catalysts with Madison Mills and Interactive Brokers chief strategist Steve Sosnick to dive into the phenomenon. To watch more expert insights and analysis on the latest market action, check out more Catalysts here.",
  'pubDate': '29/05/2025 11:30:45 UTC',
  'canonicalUrl': 'https://finance.yahoo.com/video/bitcoin-miners-powering-ai-boom-113045424.html'},
 {'title': "The real winner in GameStop's bitcoin pivot is Strategy",
  'summary': 'For a certain class of GameStop investor, chasing improbable upside is entirely the point.',
  'pubDate': '29/05/2025 10:00:51 UTC',
  'canonicalUrl': 'https://finance.yahoo.com/news/the-real-winner-in-gamestops-bitcoin-pivot-is-strategy-100051206.html'},
 {'title': 'Bitcoin Price Falls. Key Crypto Conf